# Portfolio Project - Web Scraping Football Matches From The EPL in Python

### Introduction

This project is a first part of a Machine Learning project where I will predict the winner of each football match on the English Premier League (EPL).
In this project the main goal is gathering data for a cleaned pandas dataframe via web scraping.

### Downloading and Exploring the EPL Stats Page

In [3]:
import requests

In [4]:
url = "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats"

In [5]:
page = requests.get(url)

In [7]:
page.status_code # if the code is 200 then OK

200

Inspecting the page in web browser I can see the `id` of the table-tag which contains the teams and theirs results is `results2021-202291_overall`